In [16]:
import telebot
import cv2
import numpy as np
from io import BytesIO

bot = telebot.TeleBot('yourToken')
#الكود بيقرأ البوت ويدخله

@bot.message_handler(content_types=['photo'])
def handle_photo(message):
    file_id = message.photo[-1].file_id
    file_info = bot.get_file(file_id)
    file = bot.download_file(file_info.file_path)

    np_array = np.frombuffer(file,dtype=np.uint8)
    img = cv2.imdecode(np_array,cv2.IMREAD_COLOR)

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30,30))

    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # حفظ الصورة المعالجة
    processed_image_path = "processed_image.jpg"
    cv2.imwrite(processed_image_path, img)

    # إرسال الصورة المعالجة إلى المستخدم
    processed_image = open(processed_image_path, "rb")
    bot.send_photo(message.chat.id, processed_image)

    # إرسال رسالة تؤكد معالجة الصورة
    bot.send_message(message.chat.id, "تم معالجة الصورة بنجاح!")

# بدء الاستماع على الرسائل الواردة
bot.polling()



